In [14]:
 import pymongo
import pandas as pd
from pandas import DataFrame, Series
from pymongo import MongoClient
import json
import plotly as py
py.tools.set_credentials_file(username = 'jcalcutt', api_key = '0PZ56yzpReSTHOb3Ul6k')
import cufflinks
import datetime
from bson.objectid import ObjectId

In [15]:
client = MongoClient(host = '10.50.0.94:27017')

In [16]:
db = client.twitterdb

In [17]:
collection = db.seventeenth_may

In [18]:
pipe = [{"$match": {"_id": {"$lt": ObjectId("591efa600000000000000000"), "$gt":ObjectId("591d54800000000000000000")}}},{"$project":{"Location": "$user.location", "Category": "$Bin", "tweet": "$text", "Polarity": "$compound.compound", "Subjectivity": "$subjectivity", "Date" : "$created_at"}}]

In [19]:
q = collection.aggregate(pipeline=pipe)

In [20]:
z = list(q)

In [21]:
z

[{u'Category': u'Neutral',
  u'Date': u'Thu May 18 08:37:23 +0000 2017',
  u'Location': u'West Midlands UK',
  u'Polarity': 0.0,
  u'Subjectivity': 0.0,
  u'_id': ObjectId('591d5d44c1bb431f9801ff24'),
  u'tweet': u'https://t.co/PdiRSA46yu #GE2017 #GenNext2017 #GeneralElection #VoteLabour #Labour #LabourParty #conservative #Corbyn'},
 {u'Category': u'Neutral',
  u'Date': u'Thu May 18 08:37:24 +0000 2017',
  u'Location': u'sheffield heeley',
  u'Polarity': -0.296,
  u'Subjectivity': 0.9,
  u'_id': ObjectId('591d5d44c1bb431f9801ff26'),
  u'tweet': u'#torymanifesto if this was a fair country, the media would slaughter the Tories for this savage Manifesto. #GE2017'},
 {u'Category': u'Neutral',
  u'Date': u'Thu May 18 08:37:24 +0000 2017',
  u'Location': u'England, Scotland, USA',
  u'Polarity': 0.0,
  u'Subjectivity': 0.0,
  u'_id': ObjectId('591d5d44c1bb431f9801ff29'),
  u'tweet': u'RT @theipaper: If Labour and the Lib Dems voted tactically in these seats, they could outnumber Conservative

In [22]:
dfnew = DataFrame(z)

In [23]:
dfnew['Date'] = pd.to_datetime(dfnew['Date'])

In [24]:
dfnew['Date'] = dfnew['Date'].values.astype('datetime64[h]')

In [25]:
dfnew = dfnew[dfnew['Date'] > '2017-05-11 16:00:00']

In [26]:
groupcat = dfnew.groupby(['Date', 'Category']).mean().reset_index()

In [27]:
groupcat[:10]

,Date,Category,Polarity,Subjectivity
0,2017-05-18 08:00:00,Conservative,0.058643,0.289205
1,2017-05-18 08:00:00,Cymru,-0.140500,0.000000
2,2017-05-18 08:00:00,Green,0.347554,0.448425
3,2017-05-18 08:00:00,Labour,0.053165,0.339698
4,2017-05-18 08:00:00,LibDem,0.097043,0.407013
5,2017-05-18 08:00:00,Neutral,0.074822,0.288240
6,2017-05-18 08:00:00,SNP,0.199179,0.373239
7,2017-05-18 08:00:00,UKIP,-0.014319,0.276549
8,2017-05-18 08:00:00,other,-0.083012,0.375186
9,2017-05-18 09:00:00,Conservative,0.048727,0.297860


In [28]:
groupcatconservative = groupcat.filter({'Category' : 'Conservative'})

In [29]:
a = groupcat.pivot('Date', 'Category').reset_index()

In [31]:
a[:5]

Date     Polarity                                \
Category                     Conservative     Cymru     Green    Labour   
0        2017-05-18 08:00:00     0.058643 -0.140500  0.347554  0.053165   
1        2017-05-18 09:00:00     0.048727  0.332614  0.426865  0.035210   
2        2017-05-18 10:00:00     0.069859 -0.091989  0.496673  0.037721   
3        2017-05-18 11:00:00     0.051125  0.170971  0.221262  0.089562   
4        2017-05-18 12:00:00    -0.022202 -0.057650  0.106634  0.036946   

                                                           Subjectivity  \
Category    LibDem   Neutral       SNP      UKIP     other Conservative   
0         0.097043  0.074822  0.199179 -0.014319 -0.083012     0.289205   
1         0.068981  0.067535  0.139203  0.005492 -0.006888     0.297860   
2         0.055058  0.064962  0.127660 -0.018496 -0.000444     0.298113   
3         0.053581  0.064282  0.071230 -0.019964  0.037238     0.344978   
4         0.008613  0.034673  0.095645  0.021916  0.023763     0.370854   

                                                                      \
Category     Cymru     Green    Labour    LibDem   Neutral       SNP   
0         0.000000  0.448425  0.339698  0.407013  0.288240  0.373239   
1         0.157143  0.325410  0.336523  0.237627  0.280708  0.319366   
2         0.063889  0.375914  0.354018  0.313325  0.290613  0.287912   
3         0.207792  0.321659  0.333871  0.339058  0.295933  0.289988   
4         0.200000  0.324846  0.319914  0.244683  0.315616  0.339661   

                              
Category      UKIP     other  
0         0.276549  0.375186  
1         0.252896  0.356454  
2         0.402145  0.343050  
3         0.431462  0.364085  
4         0.407924  0.310843

In [32]:
a['Polarity'][:5]

Category,Conservative,Cymru,Green,Labour,LibDem,Neutral,SNP,UKIP,other
0,0.058643,-0.140500,0.347554,0.053165,0.097043,0.074822,0.199179,-0.014319,-0.083012
1,0.048727,0.332614,0.426865,0.035210,0.068981,0.067535,0.139203,0.005492,-0.006888
2,0.069859,-0.091989,0.496673,0.037721,0.055058,0.064962,0.127660,-0.018496,-0.000444
3,0.051125,0.170971,0.221262,0.089562,0.053581,0.064282,0.071230,-0.019964,0.037238
4,-0.022202,-0.057650,0.106634,0.036946,0.008613,0.034673,0.095645,0.021916,0.023763


In [33]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 19 columns):
(Date, )                        28 non-null datetime64[ns]
(Polarity, Conservative)        28 non-null float64
(Polarity, Cymru)               25 non-null float64
(Polarity, Green)               27 non-null float64
(Polarity, Labour)              28 non-null float64
(Polarity, LibDem)              27 non-null float64
(Polarity, Neutral)             28 non-null float64
(Polarity, SNP)                 27 non-null float64
(Polarity, UKIP)                28 non-null float64
(Polarity, other)               28 non-null float64
(Subjectivity, Conservative)    28 non-null float64
(Subjectivity, Cymru)           25 non-null float64
(Subjectivity, Green)           27 non-null float64
(Subjectivity, Labour)          28 non-null float64
(Subjectivity, LibDem)          27 non-null float64
(Subjectivity, Neutral)         28 non-null float64
(Subjectivity, SNP)             27 non-null float64
(Subje

In [34]:
df2 = groupcat[groupcat.Category == 'Labour']

In [35]:
df2[:10]

,Date,Category,Polarity,Subjectivity
3,2017-05-18 08:00:00,Labour,0.053165,0.339698
12,2017-05-18 09:00:00,Labour,0.035210,0.336523
21,2017-05-18 10:00:00,Labour,0.037721,0.354018
30,2017-05-18 11:00:00,Labour,0.089562,0.333871
39,2017-05-18 12:00:00,Labour,0.036946,0.319914
48,2017-05-18 13:00:00,Labour,0.049544,0.330793
57,2017-05-18 14:00:00,Labour,0.070773,0.320520
66,2017-05-18 15:00:00,Labour,0.095257,0.314194
75,2017-05-18 16:00:00,Labour,0.083291,0.318071
84,2017-05-18 17:00:00,Labour,0.083430,0.321597


In [36]:
a.iplot(kind='line', xTitle = 'Time', yTitle = 'Sentiment', x = ['Date'], y = ['Polarity'])

In [38]:
news = pd.read_csv('c:\\Election\\twitter-scrape-master\\newsheadlines.txt', sep='|')

In [39]:
news[:3]

,timestamp,binname,website,head1,head2,head3,head4,head5,head6,head7,...,head18,head19,head20,head21,head22,head23,head24,head25,head26,head27
0,2017-05-18 09:21:45.530000,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN
1,2017-05-18 09:21:50.145000,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN
2,2017-05-18 09:21:55.163000,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN


In [40]:
news['timestamp'] = pd.to_datetime(news['timestamp'])

In [41]:
news[:3]

,timestamp,binname,website,head1,head2,head3,head4,head5,head6,head7,...,head18,head19,head20,head21,head22,head23,head24,head25,head26,head27
0,2017-05-18 09:21:45.530,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN
1,2017-05-18 09:21:50.145,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN
2,2017-05-18 09:21:55.163,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN


In [42]:
news2 = news.set_index("timestamp", drop=False)

In [43]:
news2[:2]

,timestamp,binname,website,head1,head2,head3,head4,head5,head6,head7,...,head18,head19,head20,head21,head22,head23,head24,head25,head26,head27
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-05-18 09:21:45.530,2017-05-18 09:21:45.530,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN
2017-05-18 09:21:50.145,2017-05-18 09:21:50.145,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN


In [44]:
a2 = a.set_index("Date")

In [45]:
a2[:3]

Polarity                                          \
Category            Conservative     Cymru     Green    Labour    LibDem   
Date                                                                       
2017-05-18 08:00:00     0.058643 -0.140500  0.347554  0.053165  0.097043   
2017-05-18 09:00:00     0.048727  0.332614  0.426865  0.035210  0.068981   
2017-05-18 10:00:00     0.069859 -0.091989  0.496673  0.037721  0.055058   

                                                            Subjectivity  \
Category              Neutral       SNP      UKIP     other Conservative   
Date                                                                       
2017-05-18 08:00:00  0.074822  0.199179 -0.014319 -0.083012     0.289205   
2017-05-18 09:00:00  0.067535  0.139203  0.005492 -0.006888     0.297860   
2017-05-18 10:00:00  0.064962  0.127660 -0.018496 -0.000444     0.298113   

                                                                       \
Category                Cymru     Green    Labour    LibDem   Neutral   
Date                                                                    
2017-05-18 08:00:00  0.000000  0.448425  0.339698  0.407013  0.288240   
2017-05-18 09:00:00  0.157143  0.325410  0.336523  0.237627  0.280708   
2017-05-18 10:00:00  0.063889  0.375914  0.354018  0.313325  0.290613   

                                                   
Category                  SNP      UKIP     other  
Date                                               
2017-05-18 08:00:00  0.373239  0.276549  0.375186  
2017-05-18 09:00:00  0.319366  0.252896  0.356454  
2017-05-18 10:00:00  0.287912  0.402145  0.343050

In [46]:
a2['Polarity'][:5]

Category,Conservative,Cymru,Green,Labour,LibDem,Neutral,SNP,UKIP,other
Date,,,,,,,,,
2017-05-18 08:00:00,0.058643,-0.140500,0.347554,0.053165,0.097043,0.074822,0.199179,-0.014319,-0.083012
2017-05-18 09:00:00,0.048727,0.332614,0.426865,0.035210,0.068981,0.067535,0.139203,0.005492,-0.006888
2017-05-18 10:00:00,0.069859,-0.091989,0.496673,0.037721,0.055058,0.064962,0.127660,-0.018496,-0.000444
2017-05-18 11:00:00,0.051125,0.170971,0.221262,0.089562,0.053581,0.064282,0.071230,-0.019964,0.037238
2017-05-18 12:00:00,-0.022202,-0.057650,0.106634,0.036946,0.008613,0.034673,0.095645,0.021916,0.023763


In [47]:
news3 = news2.reindex(a2.index, method='nearest')

In [51]:
news3[:5]

,timestamp,binname,website,head1,head2,head3,head4,head5,head6,head7,...,head18,head19,head20,head21,head22,head23,head24,head25,head26,head27
Date,,,,,,,,,,,,,,,,,,,,,
2017-05-18 08:00:00,2017-05-18 09:21:45.530,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN
2017-05-18 09:00:00,2017-05-18 09:21:45.530,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN
2017-05-18 10:00:00,2017-05-18 09:23:32.691,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN
2017-05-18 11:00:00,2017-05-18 12:23:32.555,headlines,https://www.bbc.co.uk/news/election/2017,May's manifesto 'for mainstream Britain',Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Former Welsh First Minister Morgan dies,People asking Google who to vote for,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,Poll tracker: See how the parties are lining up,Reality Check: Could legalising cannabis raise...,'If I were in charge I would...',Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",Johnson criticised over whisky comment,NaN
2017-05-18 12:00:00,2017-05-18 12:23:32.555,headlines,https://www.bbc.co.uk/news/election/2017,May's manifesto 'for mainstream Britain',Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Former Welsh First Minister Morgan dies,People asking Google who to vote for,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,Poll tracker: See how the parties are lining up,Reality Check: Could legalising cannabis raise...,'If I were in charge I would...',Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",Johnson criticised over whisky comment,NaN


In [52]:
data2 = pd.concat([news3, a2.Polarity], axis=1)

In [105]:
sLength = len(data2['other'])

In [53]:
data2['e'] = pd.Series(1, index=data2.index)

In [54]:
data2[:5]

,timestamp,binname,website,head1,head2,head3,head4,head5,head6,head7,...,Conservative,Cymru,Green,Labour,LibDem,Neutral,SNP,UKIP,other,e
Date,,,,,,,,,,,,,,,,,,,,,
2017-05-18 08:00:00,2017-05-18 09:21:45.530,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,0.058643,-0.140500,0.347554,0.053165,0.097043,0.074822,0.199179,-0.014319,-0.083012,1
2017-05-18 09:00:00,2017-05-18 09:21:45.530,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,0.048727,0.332614,0.426865,0.035210,0.068981,0.067535,0.139203,0.005492,-0.006888,1
2017-05-18 10:00:00,2017-05-18 09:23:32.691,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,0.069859,-0.091989,0.496673,0.037721,0.055058,0.064962,0.127660,-0.018496,-0.000444,1
2017-05-18 11:00:00,2017-05-18 12:23:32.555,headlines,https://www.bbc.co.uk/news/election/2017,May's manifesto 'for mainstream Britain',Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Former Welsh First Minister Morgan dies,People asking Google who to vote for,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',...,0.051125,0.170971,0.221262,0.089562,0.053581,0.064282,0.071230,-0.019964,0.037238,1
2017-05-18 12:00:00,2017-05-18 12:23:32.555,headlines,https://www.bbc.co.uk/news/election/2017,May's manifesto 'for mainstream Britain',Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Former Welsh First Minister Morgan dies,People asking Google who to vote for,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',...,-0.022202,-0.057650,0.106634,0.036946,0.008613,0.034673,0.095645,0.021916,0.023763,1


In [55]:
import plotly.graph_objs as go

In [56]:
data2.iplot(kind='line', xTitle = 'Time', yTitle = 'Sentiment', y = ['Labour', 'Conservative', 'Green'])

In [57]:
a2['Polarity'][:5]

Category,Conservative,Cymru,Green,Labour,LibDem,Neutral,SNP,UKIP,other
Date,,,,,,,,,
2017-05-18 08:00:00,0.058643,-0.140500,0.347554,0.053165,0.097043,0.074822,0.199179,-0.014319,-0.083012
2017-05-18 09:00:00,0.048727,0.332614,0.426865,0.035210,0.068981,0.067535,0.139203,0.005492,-0.006888
2017-05-18 10:00:00,0.069859,-0.091989,0.496673,0.037721,0.055058,0.064962,0.127660,-0.018496,-0.000444
2017-05-18 11:00:00,0.051125,0.170971,0.221262,0.089562,0.053581,0.064282,0.071230,-0.019964,0.037238
2017-05-18 12:00:00,-0.022202,-0.057650,0.106634,0.036946,0.008613,0.034673,0.095645,0.021916,0.023763


In [58]:
df3 = data2

In [59]:
df3

,timestamp,binname,website,head1,head2,head3,head4,head5,head6,head7,...,Conservative,Cymru,Green,Labour,LibDem,Neutral,SNP,UKIP,other,e
Date,,,,,,,,,,,,,,,,,,,,,
2017-05-18 08:00:00,2017-05-18 09:21:45.530,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,0.058643,-0.140500,0.347554,0.053165,0.097043,0.074822,0.199179,-0.014319,-0.083012,1
2017-05-18 09:00:00,2017-05-18 09:21:45.530,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,0.048727,0.332614,0.426865,0.035210,0.068981,0.067535,0.139203,0.005492,-0.006888,1
2017-05-18 10:00:00,2017-05-18 09:23:32.691,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,0.069859,-0.091989,0.496673,0.037721,0.055058,0.064962,0.127660,-0.018496,-0.000444,1
2017-05-18 11:00:00,2017-05-18 12:23:32.555,headlines,https://www.bbc.co.uk/news/election/2017,May's manifesto 'for mainstream Britain',Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Former Welsh First Minister Morgan dies,People asking Google who to vote for,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',...,0.051125,0.170971,0.221262,0.089562,0.053581,0.064282,0.071230,-0.019964,0.037238,1
2017-05-18 12:00:00,2017-05-18 12:23:32.555,headlines,https://www.bbc.co.uk/news/election/2017,May's manifesto 'for mainstream Britain',Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Former Welsh First Minister Morgan dies,People asking Google who to vote for,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',...,-0.022202,-0.057650,0.106634,0.036946,0.008613,0.034673,0.095645,0.021916,0.023763,1
2017-05-18 13:00:00,2017-05-18 12:23:32.555,headlines,https://www.bbc.co.uk/news/election/2017,May's manifesto 'for mainstream Britain',Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Former Welsh First Minister Morgan dies,People asking Google who to vote for,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',...,0.013421,0.125456,0.068585,0.049544,0.172803,0.025591,0.058623,-0.046303,-0.015825,1
2017-05-18 14:00:00,2017-05-18 15:23:32.562,headlines,https://www.bbc.co.uk/news/election/2017,May's manifesto for 'mainstream Britain',Reaction to Conservative manifesto,At-a-glance guide: Conservative manifesto,Tory care plan: Why many will pay more,Former Welsh First Minister Morgan dies,Politicians in thick of mixing fact and fiction,People asking Google who to vote for,...,0.026474,-0.183125,-0.000300,0.070773,0.032962,0.052229,0.095161,-0.036046,-0.070490,1
2017-05-18 15:00:00,2017-05-18 15:23:32.562,headlines,https://www.bbc.co.uk/news/election/2017,May's manifesto for 'mainstream Britain',Reaction to Conservative manifesto,At-a-glance guide: Conservative manifesto,Tory care plan: Why many will pay more,Former Welsh First Minister Morgan dies,Politicians in thick of mixing fact and fiction,People asking Google who to vote for,...,0.030932,-0.138786,0.208594,0.095257,0.001147,0.053491,0.081995,-0.032348,-0.021608,1
2017-05-18 16:00:00,2017-05-18 15:23:32.562,headlines,https://www.bbc.co.uk/news/election/2017,May's manifesto for 'mainstream Britain',Reaction to Conservative ma

In [60]:
for row in df3.values:
    #print row
    for num in row:
        val = type(num)
        #print val
        if val == float:
            if val
         
            

SyntaxError: invalid syntax (<ipython-input-60-b36530050db7>, line 7)

## To create DF containg only entries where sentiment is > 0.4

In [61]:
dftest = DataFrame()
#dftest['timestamp'] = a2.index.values
#dftest = dftest.set_index("timestamp", drop=False)

In [62]:
dftest[:5]

""


In [63]:
# Create Dataframe for headlines where sentiment value is over a certain limit. with timestamp
# Use this for annotations

i = 0
for row in a.Polarity.values:
    #print row
    for num in row:
        #print num
        if num > 0.4:
            print num
            print i
        #append headlines to new dataframe with timestamp
            dftest = dftest.append(news3[i-1:i])
        else:
            i
    i+=1
        

0.426864705882
1
0.4966734375
2
0.527075
16
0.4404
16
0.44212
17
0.5859
19
0.404731325301
23


##### Table that contains headlines for sentiment over 0.4

In [64]:
dftest[:]

,timestamp,binname,website,head1,head2,head3,head4,head5,head6,head7,...,head18,head19,head20,head21,head22,head23,head24,head25,head26,head27
Date,,,,,,,,,,,,,,,,,,,,,
2017-05-18 08:00:00,2017-05-18 09:21:45.530,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN
2017-05-18 09:00:00,2017-05-18 09:21:45.530,headlines,https://www.bbc.co.uk/news/election/2017,Tory manifesto pledge on social care,Tory care plan: Why many will pay more,Election 2017: latest updates,Former Welsh First Minister Morgan dies,Tim Farron: Public must decide on Brexit,McCluskey 'now optimistic Labour can win',At-a-glance summary: Lib Dem manifesto,...,Kuenssberg: May's bid to be tough not cruel,Why education could determine how you vote,A tale of two election campaigns,Reality Check: Could legalising cannabis raise...,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,"Retro games, period poverty and bridge tolls",What does Alliance for Green Socialism stand for?,Johnson criticised over whisky comment,NaN
2017-05-18 23:00:00,2017-05-19 00:23:33.058,headlines,https://www.bbc.co.uk/news/election/2017,Parties clash with UKIP in live debate,Question Time and Tory manifesto reaction,May's manifesto for 'mainstream Britain',At-a-glance guide: Conservative manifesto,Tory care plan: Why many will pay more,Tory social care plans spark criticism,Former Welsh First Minister Morgan dies,...,Is Theresa May a 'Red Tory'?,Guide to the parties: Green Party,Poll tracker: See how the parties are lining up,Whose election promise adds up for the NHS?,'If I were in charge I would...',Revolutionary bid for election votes,Should 16 and 17-year-olds get the vote?,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,NaN
2017-05-18 23:00:00,2017-05-19 00:23:33.058,headlines,https://www.bbc.co.uk/news/election/2017,Parties clash with UKIP in live debate,Question Time and Tory manifesto reaction,May's manifesto for 'mainstream Britain',At-a-glance guide: Conservative manifesto,Tory care plan: Why many will pay more,Tory social care plans spark criticism,Former Welsh First Minister Morgan dies,...,Is Theresa May a 'Red Tory'?,Guide to the parties: Green Party,Poll tracker: See how the parties are lining up,Whose election promise adds up for the NHS?,'If I were in charge I would...',Revolutionary bid for election votes,Should 16 and 17-year-olds get the vote?,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,NaN
2017-05-19 02:00:00,2017-05-19 03:23:32.967,headlines,https://www.bbc.co.uk/news/election/2017,Parties clash with UKIP in live debate,No timetable for Tory migration pledge,May's manifesto for 'mainstream Britain',At-a-glance guide: Conservative manifesto,Tory care plan: Why many will pay more,Tory social care plans spark criticism,Former Welsh First Minister Morgan dies,...,Is Theresa May a 'Red Tory'?,Guide to the parties: Green Party,Poll tracker: See how the parties are lining up,Whose election promise adds up for the NHS?,'If I were in charge I would...',Revolutionary bid for election votes,Should 16 and 17-year-olds get the vote?,Lib Dem manifesto: 'We can have change',IndyRef2 battle and the general election,NaN
2017-05-19 04:00:00,2017-05-19 03:23:32.967,headlines,https://www.bbc.co.uk/news/election/2017,Parties clash with UKIP in 

In [96]:
df5 = DataFrame()

In [97]:
df5["headlines"] = '1) '+data2["head1"].map(str) + ', '+  '2) '+data2["head2"] + ', ' + '3) '+data2['head3']

In [98]:
df5[:3]

,headlines
Date,
2017-05-18 08:00:00,"1) Tory manifesto pledge on social care, 2) To..."
2017-05-18 09:00:00,"1) Tory manifesto pledge on social care, 2) To..."
2017-05-18 10:00:00,"1) Tory manifesto pledge on social care, 2) To..."


In [105]:
import plotly.plotly as py
import plotly.graph_objs as go


import numpy as np



# Create traces
trace0 = go.Scatter(
    x = data2.index,
    y = data2.Conservative,
    mode = 'lines',
    name = 'Conservative',
    #text = data2.head1,
    marker=dict(color = 'rgb(65,105,225)') 
                
)


trace1 = go.Scatter(
    x = data2.index,
    y = data2.Labour,
    mode = 'lines',
    name = 'Labour',
    marker=dict(color = 'rgb(139,0,0)') 
    
)
trace2 = go.Scatter(
    x = data2.index,
    y = data2.LibDem,
    mode = 'lines',
    name = 'LibDem',
    marker=dict(color = 'rgb(255,165,0)') 
    
)
trace3 = go.Scatter(
    x = data2.index,
    y = data2.Green,
    mode = 'lines',
    name = 'Green',
    marker=dict(color = 'rgb(34,139,34)')     
)

trace4 = go.Scatter(
    x = data2.index,
    y = data2.UKIP,
    mode = 'lines',
    name = 'UKIP',
    marker=dict(color = 'rgb(148,0,211)') 
)

trace5 = go.Scatter(
    x = data2.index,
    y = data2.SNP,
    mode = 'lines',
    name = 'SNP',
    marker=dict(color = 'rgb(255,255,0)') 
    
)

trace6 = go.Scatter(
    x = data2.index,
    y = data2.e,
    mode = 'markers',
    name = 'News headlines',
    text = df5.headlines
    
)
layout = go.Layout(
    
    xaxis=dict(
        title='Time',
        
    ),
    yaxis=dict(
        title='Sentiment (+1 -> -1)',
        
    )
)
data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6]

py.iplot(data, xTitle = 'Time', yTitle = 'Sentiment')

In [74]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

fig = {
    'data': [
                {
             
            'y': data2.Conservative,
            #'text' : dftest.head1,
            'mode' : 'lines+markers+text',
            
            
            
                        
       }
   
       ],
   'layout': {
       'xaxis': {'title': 'time'},
       'yaxis': {'title': 'Subjectivity'},
       'annotations' :[
       dict(
           x= 30,
           y = 1,
           #xref='x',
           #yref='y',
           text= data.loc[row].head1,
           showarrow=True,
           arrowhead=7,
           ax=0,
           ay=-40
       )for row in data.index.values ]

    }
}



NameError: name 'data' is not defined

In [ ]:
py.iplot(fig)

In [ ]:
for row in data2:
    if data2['Conservative'] > 0.2:
        print head1
    

In [ ]:
annotations1 = [dict(
            x=xi-0.2,
            y=yi,
            text=data.head1),
            xanchor='auto',
            yanchor='bottom',
            showarrow=False,
        ) for xi, yi in zip(xcoord, y1)]

In [ ]:
dfnew2 = dfnew.tail(10000) 

In [ ]:
df3 = dfnew2[(dfnew2['Polarity'] == 0)]

In [ ]:
df3[df3['Polarity'] <> 0].describe()

In [ ]:
df3.describe()

In [ ]:
dfnew.describe()

In [ ]:
sentences = ['The movie was the worst movie', 'It was the worst acting by the actors']


In [ ]:
from senti_classifier import senti_classifier


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [ ]:
try:
    import cPickle as pickle
except ImportError:
    import _pickle as pickle
import senti_classifier


In [ ]:
sentences = ' xD '


In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
vs = analyzer.polarity_scores(sentences)

In [ ]:
print vs
 

In [ ]:
SentimentIntensityAnalyzer().polarity_scores(sentences)

In [ ]:
TextBlob(sentences).sentiment

In [ ]:
import geonamescache


In [ ]:
gc = geonamescache.GeonamesCache()

In [ ]:
countries = gc.get_cities()

In [ ]:
dc = DataFrame(countries)

In [ ]:
dc

countries['293294']

In [ ]:
countrie

In [ ]:
list = [v['name'] for k,v in countries.items() if v['countrycode'] == 'GB']

In [ ]:
list

In [ ]:
countries[u'countrycode']

In [ ]:
def get_closest_match(x, list_strings):
 
    best_match = None
    highest_jw = 0
 
    for current_string in list_strings:
        current_score = jellyfish.jaro_winkler(x, current_string)
 
    if(current_score > highest_jw):
        highest_jw = current_score
        best_match = current_string
 
    return best_match